In [1]:
import os
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn import tree
import numpy as np
import pandas as pd
import pickle
from random import sample 
from sklearn.tree import DecisionTreeRegressor

# Read Dataset

In [2]:
path = "../predict-responsibly/data/"
dataset = "adult"
protected_feature = "sex"
K = 1
db = np.load(os.path.join(path,dataset,"{}{}_{}.npz".format(dataset,K,protected_feature)))

## partition X according to f0

In [3]:
x_val = db["x_train"]
y_val = db["y_train"]
x_test = db["x_test"]
y_test = db["y_test"]

In [4]:
def res(p, y):
    return y * ((p>=0.1)/(p + 1e-20) + (p<0.1) * (20 - 100  * p)) +\
(1-y) * ((p < 0.9)/(1 - p + 1e-20) + (p>=0.9) * (100 * p - 80))

In [26]:
max_T = 100
thresh = 1e-4 # Hyper-parameter

In [ ]:
best_epoch, best_acc = -1,0
# get the validation index

valid_index = np.arange(len(x_val))
#use budget % of the validation set for training
valid_index = sample(list(valid_index), int(len(valid_index) * 0.1))


# get the idxs1 and idxs2
idxs1 = sample(list(valid_index), int(len(valid_index) * 0.7))
idxs2 = list(set(valid_index) - set(idxs1))

# save the coefficients
coeffs = []
for t in range(max_T):
    # probs_heldout = sess_run(tf.nn.sigmoid(logits), x_val[idxs2], latent_val[idxs2], sess)
    probs_heldout = db["ydm_train"][idxs2]
    heldout_loss = np.mean(-y_val[idxs2] * np.log(probs_heldout + 1e-20) - (1-y_val[idxs2]) * np.log(1-probs_heldout + 1e-20))
    heldout_acc =  np.mean((probs_heldout>0.5)==y_val[idxs2])
    # probs = sess_run(tf.nn.sigmoid(logits), x_val, latent_val ,sess)
    probs = db["ydm_train"]
    val_loss = np.mean(-y_val * np.log(probs + 1e-20) - (1 - y_val) * np.log(1 - probs + 1e-20))
    val_acc = np.mean((probs > 0.5) == y_val)
    if heldout_acc > best_acc:
        best_epoch = t
        best_acc = heldout_acc
        best_logits = logits
    delta = res(probs,y_val)
    residual = probs - y_val
    # define the control
    control = db["ydm_train"]>0.5
    for i in range(3):
        if i==0: 
            # X_1
            temp_s = control[idxs1]
            temp_s_heldout = control[idxs2]
        elif i ==1：
            # X_0
            temp_s = 1- control[idxs1]
            temp_s_heldout = 1- control[idxs2]
        else:
            # X
            temp_s = np.ones_like(control[idxs1])
            temp_s_heldout = np.ones_like(control[idxs2])
        # get the fresh sample for training
        samples1 = np.where(temp_s == 1)[0]
        samples2 = np.where(temp_s_heldout == 1)[0]
        # train the regression model 
        clf = DecisionTreeRegressor
        # 如果要把protected feature 去掉 可以考率生成一个新的x_train、valid
        clf.fit(x_val[idxs1][samples1],delta[idxs1][samples1])
        clf_prediction = clf.predict(x_val[idxs2][samples2])
        corr = np.mean(clf_prediction * residual[idxs2][samples2])
        print(t, i, corr)
        if corr > 1e-4:
            coeffs.append(clf.coef_)
            h = (tf.matmul(latent_ph, tf.constant(np.expand_dims(clf.coef_,-1),
                                                    dtype=tf.float32))[:,0] + clf.intercept_)
            logits -= .1 * h * s
            break